<center>
    <table>
        <tr>
            <th> <img src="figures/ufrr_logo.png" width=75px> </th>
            <th> <img src="figures/dcc.jpg" width=100px> </th>
        </tr>
    <table>
    <font size=4>
    Universidade Federal de Roriama
    <br>
    Departamento de Ciência da Computação
    <br>
    Professor: Filipe Dwan Pereira
    <br>
    Código da disciplina: DCC305
    <br>
    Período: 2019.2
    </font>
</center>

## Disclaimer

Esta aula é uma adaptação do capítulo 3 do livro:

* Phillips, Dusty. Python 3 Object-oriented Programming - Unleash the power of Python 3 objects. "Packt Publishing", 2015. Second Edition.

Nesta aula aprenderemos:

* O básico sobre herança
* Usar herança a partir de built-ins
* Herança Múltipla;
* Polimorfismos e duck typing;

## Herança Básica

* Tecnicamente, todas as classes que usamos já usam herança, uma vez que elas herdam da classe *object*;
    * Os comportamentos fornecidos por essa classe são aqueles com underscore duplo;
* A classe object permite que o python trate todos os objetos da mesma maneira, isto é, como um objeto;
* Até agora ao clicar nossas classes não explicitamos que herdamos de object, mas podemos, conforme segue:

In [ ]:
class MySubClass(object):
    pass

* Isso é **herança**;
* Chamamos de superclasse ou classe pai a classe a qual herdamos e de subclasse ou classe filha a classe que herda.
    * No caso acima, object é a superclasse e MySubClasse a subclasse;
* Para realizar a herança em python basta colocar o nome da superclasse a qual se quer herdar dentro dos parênteses da definidação da subclasse;

* O uso mais simples da herança é adicionar funcionalidade a uma classe existente.
* Para ilustrar, veja um simples exemplo de uma classe base que gerencia contados:

In [16]:
class Contact:
    all_contacts = []
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

* Neste exemplo usamos o que chamamos de variável de classe;
    * A lista *all_contacts* é uma variável de classe porque é compartilhada por todos objetos da classe *Contact*.
* Agora imagine que existem contatos de fornecedores que fazemos pedidos;
    * Poderíamos pensar em adicionar um método **order** na classe Contact, mas isso faria que contatos familiares também tivessem esse método;
    * Vamos criar uma classe Fornecedores (*Supplier* em inglês) que além de herdar de Contact, também possui o método *order*:

In [17]:
class Supplier(Contact):
    def order(self, pedido):
        print("Pedido para '{}': "
            "'{}'".format(self.name, pedido))

In [18]:
c1 = Contact('Filipe', 'filipe@gmail.com')
f1 = Supplier('Amazon', 'amazon@gmail.com')

* Note que o objeto f1 possui nome e email pois herda da classe Contact:

In [22]:
print(c1.name, c1.email)
print(f1.name, f1.email)

Filipe filipe@gmail.com
Amazon amazon@gmail.com


* Perceba ainda que apenas o objeto f1 é capaz de efetuar pedidos:

In [19]:
f1.order('Preciso de 30 caixas de canetas')

Pedido para 'Amazon': 'Preciso de 30 caixas de canetas'


In [23]:
c1.order('Preciso de 30 caixas de canetas')

AttributeError: 'Contact' object has no attribute 'order'

* Agora perceba como o atributo *all_contacts* é compartilhado por ambos objetos:

In [20]:
c1.all_contacts

[<__main__.Contact at 0x7f966bace320>, <__main__.Supplier at 0x7f966bace358>]

In [24]:
f1.all_contacts

[<__main__.Contact at 0x7f966bace320>, <__main__.Supplier at 0x7f966bace358>]

## Herdando de built-ins

* Um uso interessante desse tipo de herança é adicionar funcionalidade aos recursos built-ins do python;
* Para exemplo, podemos criar uma classe *ContactList* que adiciona um método para fazer busca de contatos em uma lista padrão do python;
    * Para tanto, herdaremos de *list*:
* Após isso, iremos usar a ContectList ao invés de uma lista padrão para iniciazar nossa variável de classe *all_contacts* (linha 10 do código abaixo):

In [61]:
class ContactList(list):    
    def search(self, name):
        matching_contacts = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts

class Contact():
    all_contacts = ContactList()
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

* Para testar nosso exemplo, vamos criar três contatos:

In [62]:
c1 = Contact('Fulano da Silva', 'fulano1@dominio.com')
c2 = Contact('Fulano Sousa', 'fulano2@dominio.com')
c3 = Contact('Ciclano Pereira', 'ciclano3@dominio.com')

* Agora vamos fazer uma busca por contatos com nome *Fulano*;

In [64]:
Contact.all_contacts.search('Fulano')

[<__main__.Contact at 0x7f966b5a8be0>, <__main__.Contact at 0x7f966b5a8c18>]

* Bem, existem dois objetos com esse nome;
    * Vamos apresentá-los:

In [63]:
[c.name for c in Contact.all_contacts.search('Fulano')]

['Fulano da Silva', 'Fulano Sousa']

* Note que a sintaxe built-in [ ] é equivalante a *list()*:

In [65]:
[] == list()

True

* Assim sendo, quando fazemos *variavel = [ ]*, estamos instanciando um objeto da classe *list*;
* Ainda, a classe *list* herda de object, como podemos ver abaixo:

In [66]:
isinstance([], object)

True

* Como um segundo exemplo, vamos herdar agora da classe *dict*;
* Iremos adicionar uma funcionalidade para verificar qual é a chave mais longa do dicionário:

In [78]:
class LongNameDict(dict):
    def longest_key(self):
        longest = None
        for key in self.keys():
            if not longest or len(key) > len(longest):
                longest = key
        return longest

In [79]:
longkeys = LongNameDict()
longkeys['hello'] = 1
longkeys['longest yet'] = 5
longkeys['hello2'] = 'world'
longkeys.longest_key()

'longest yet'

## Sobrescrevendo métodos

* Além de adicionar funcionalidades ao utilizar herença, podemos também modificá-las;
    * Isto é, podemos herdar comportamentos e modificá-los;
* Por exemplo, imagine que queremos adicionar o atributo phone_number nos contatos dos nossos amigos mais próximos;
    * Para tanto, podemos criar uma classe Friend que herda de Contact e sobrescreve o construtor:

In [80]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone

* Observe 2 problemas com o código acima:
    1. Repetimos código o que pode tornar o processo de manutenção do sistema ruim;
    2. Esquecemos de adicionar o contado do amigo na variável de classe *all_contacts*;
* O que nós realmente precisamos é executar o \_\_init\_\_ da classe Contact e apenas adicionar o telefone do contato;
* Para tanto, podemos usar a palavra chave **super**, que nos permite invocar métodos da superclasse diretamente:

In [83]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        super().__init__(name, email)
        self.phone = phone

* Vamos criar um amigo, apenas para testar a classe:

In [84]:
friend1 = Friend('Beltrano Ferreira', 'beltrano@dominio.com', '2121-3131')

In [85]:
friend1.phone

'2121-3131'

## Herança Múltipla

* A herança múltipla é um assunto delicado.
